In [20]:
###CONFIGURAÇÕES INICIAIS###

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, monotonically_increasing_id, when, concat_ws
from pyspark.sql.types import DoubleType
from datetime import datetime, timedelta

# Inicializando a sessão do Spark
spark = SparkSession.builder \
    .appName("Gold") \
    .getOrCreate()


# Caminho para salvar o arquivo
file_path = 'C:/Users/yurib/pjf-engenharia-de-dados/algo/'

In [21]:
### CARREGANDO TABELAS DA CAMADA SILVER ###

# Carregar tabelas da camada Silver
datas = spark.table('dim_data')
entregas = spark.table('dim_entregas')
pedidos = spark.table('fato_pedidos')
transacoes = spark.table('dim_transacoes')
localizacoes = spark.table('dim_localizacoes')

C:\Users\yurib\AppData\Local\Temp\ipykernel_7804\3084060704.py:6: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  transacoes = pd.read_csv(file_path+'silver/dim_transacao.csv')


In [ ]:
### CRIAR TABELA FATO ###

# Processamento de distâncias usando Spark SQL
entregas_pedidos_vendedor = entregas.join(localizacoes.withColumnRenamed('SK_Localizacoes', 'en_origem'), on='en_origem', how='left') \
                                    .join(localizacoes.withColumnRenamed('SK_Localizacoes', 'en_destino'), on='en_destino', how='left') \
                                    .select('SK_Pedidos', 'latitude as latitude_vendedor', 'longitude as longitude_vendedor',
                                            'latitude1 as latitude_cliente', 'longitude1 as longitude_cliente')

# Função para calcular a distância usando Haversine em Spark SQL
@udf(returnType=DoubleType())
def haversine_udf(lat1, lon1, lat2, lon2):
    R = 6371.0  # Raio da Terra em quilômetros

    # Converter graus para radianos
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(math.radians, [lat1, lon1, lat2, lon2])

    # Diferença das latitudes e longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Aplicar fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Calcular a distância
    distance = R * c
    return distance

In [ ]:
# Aplicar a função haversine_udf para calcular a distância em km
entregas_pedidos_vendedor = entregas_pedidos_vendedor.withColumn('distancia_km', haversine_udf(
    'latitude_vendedor', 'longitude_vendedor', 'latitude_cliente', 'longitude_cliente'))

# Selecionar colunas relevantes para o fato de pedidos
fato_pedidos = pedidos.join(entregas_pedidos_vendedor, on='SK_Pedidos', how='left') \
                      .select('SK_Pedidos', 'preco', 'frete', 'volume', 'massa', 'distancia_km',
                              'ped_diasentrega', 'ped_diasprazo')

In [27]:
## GERAR DIMENSIONAIS ###

# Remover colunas indesejadas e salvar como tabelas Spark
datas = datas.drop('Unnamed: 0')
entregas = entregas.drop('Unnamed: 0')
transacoes = transacoes.drop('Unnamed: 0')
localizacoes = localizacoes.drop('Unnamed: 0')

datas.write.mode('overwrite').saveAsTable('dim_data')
fato_pedidos.write.mode('overwrite').saveAsTable('fato_pedidos')
entregas.write.mode('overwrite').saveAsTable('dim_entregas')
transacoes.write.mode('overwrite').saveAsTable('dim_transacoes')
localizacoes.write.mode('overwrite').saveAsTable('dim_localizacoes')

In [ ]:
# Finalizando a sessão Spark
spark.stop()